In [30]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [33]:
MODEL_PATH = '/models/nlp_models'

In [34]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###**Text preprocessing functions**

In [35]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [36]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 20000)

In [37]:
MAX_LEN = 160
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
random.shuffle(sents_data)
sents_data = sents_data[:15000]  # make dataset size 15000 for ease of computation

In [38]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sents_data)
tokenizer.word_index['<PAD>'] = 0
last_idx = len(tokenizer.word_index) + 1
tokenizer.word_index['<SOS>'] = last_idx

In [39]:
temp = [len(each.split()) for each in sents_data]
sorted(temp)[-1]

36

### **building the dataset**

In [40]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m + 1):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [41]:
shuffle_pairs = create_shuffles(sents_data, 5)

In [42]:
shuffle_pairs[0][0], shuffle_pairs[0][1]

(' ایالات متحده روسیه را به قطع همکاریهای آن کشور در پروژه توسعه اتمی ایران تشویق کرد',
 {'ایالات اتمی در متحده پروژه همکاریهای به کشور قطع کرد تشویق را توسعه ایران روسیه آن',
  'ایالات قطع روسیه در را آن متحده به همکاریهای ایران کرد اتمی کشور توسعه تشویق پروژه',
  'را کشور آن اتمی در پروژه توسعه ایران ایالات تشویق متحده قطع روسیه همکاریهای به کرد',
  'قطع پروژه روسیه اتمی تشویق متحده کشور آن ایران کرد به در ایالات توسعه را همکاریهای',
  'همکاریهای ایالات در آن به توسعه ایران کشور کرد را پروژه متحده قطع تشویق اتمی روسیه',
  'کشور ایالات اتمی آن پروژه را کرد همکاریهای توسعه روسیه در متحده تشویق ایران به قطع'})

In [43]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [44]:
df = create_dataframe(shuffle_pairs)
df

,Shuffled,Original
0,را کشور آن اتمی در پروژه توسعه ایران ایالات تش...,ایالات متحده روسیه را به قطع همکاریهای آن کشو...
1,ایالات اتمی در متحده پروژه همکاریهای به کشور ق...,ایالات متحده روسیه را به قطع همکاریهای آن کشو...
2,قطع پروژه روسیه اتمی تشویق متحده کشور آن ایران...,ایالات متحده روسیه را به قطع همکاریهای آن کشو...
3,همکاریهای ایالات در آن به توسعه ایران کشور کرد...,ایالات متحده روسیه را به قطع همکاریهای آن کشو...
4,کشور ایالات اتمی آن پروژه را کرد همکاریهای توس...,ایالات متحده روسیه را به قطع همکاریهای آن کشو...
...,...,...
88887,اتیوپی وزیراطلاعات درمصاحبه‌ای شوند اظهارداشت ...,وزیراطلاعات اتیوپی روزچهارشنبه درمصاحبه‌ای با...
88888,وارد درمصاحبه‌ای روزچهارشنبه با ندارند اتیوپی ...,وزیراطلاعات اتیوپی روزچهارشنبه درمصاحبه‌ای با...
88889,وزیراطلاعات درمصاحبه‌ای موگادیشو صدای ندارند ا...,وزیراطلاعات اتیوپی روزچهارشنبه درمصاحبه‌ای با...
88890,روزچهارشنبه وارد با صدای ندارند شوند اظهارداشت...,وزیراطلاعات اتیوپی روزچهارشنبه درمصاحبه‌ای با...


In [45]:
def create_dataset(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  dataset = df['Shuffled'].values
  labels  = df['Original'].values

  train_data   = dataset[:count]
  train_labels = labels[:count]
  test_data   = dataset[count:]
  test_labels = labels[count:]

  return ((train_data, train_labels), (test_data, test_labels))




In [54]:
train_set, test_set = create_dataset(df)
X_train_text, Y_train_text = train_set
X_test_text, Y_test_text = test_set

###**making dataset ready for training**

In [55]:
sos_index = last_idx
def shift_output_sequence(seq_list):
  for seq in seq_list:
    seq.insert(0, sos_index)
  return seq_list


In [56]:
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
Y_train_seq = tokenizer.texts_to_sequences(Y_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)
Y_test_seq  = tokenizer.texts_to_sequences(Y_test_text)

In [57]:
MAX_SEQ_LEN = 40  
X_train = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN)
Y_train = keras.preprocessing.sequence.pad_sequences(Y_train_seq, maxlen=MAX_SEQ_LEN)

X_test  = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN)
Y_test  = keras.preprocessing.sequence.pad_sequences(Y_test_seq, maxlen=MAX_SEQ_LEN)

X_train_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_train_seq), maxlen=MAX_SEQ_LEN)
X_test_decoder  = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_test_seq),  maxlen=MAX_SEQ_LEN)

In [58]:
X_train_decoder[:2], X_test_decoder[:2]

(array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 20482,   205,   190,    88,
             6,     3,   771,  3119,    23,    30,     2,  2601,  1009,
            57,    20,  1408,    14],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 20482,   205,   190,    88,
             6,     3,   771,  3119,    23,    30,     2,  2601,  1009,
            57,    20,  1408,    14]], dtype=int32),
 array([[    0,     0,     0,     0,     0,     0,     0,     0, 20482,
             2,   700,    32,   919,   221,   285,     7,  2486,  5685,
           241,  7604, 18358,   470, 18359,    43, 18360,     2,  4429,
             6,     2,  1101,  9242,  5685,   241,     9,    41,    45,
            50,  3282,  1455,    18],
       

In [59]:
X_train[0], Y_train[0]

(array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    6,   30,   23,   57,    2, 2601, 1009,   20,  205,
        1408,  190,  771,   88, 3119,    3,   14], dtype=int32),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  205,  190,   88,    6,    3,  771, 3119,   23,   30,
           2, 2601, 1009,   57,   20, 1408,   14], dtype=int32))

In [60]:
VOCAB_SIZE = len(tokenizer.word_index)
VOCAB_SIZE

20482

###**Encoder-Decoder network**

In [74]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

encoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 1,output_dim=encoder_embedding_size, input_length=MAX_SEQ_LEN, mask_zero=True)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, return_state=True)(encoder_embedding)

encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

decoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 2, output_dim=decoder_embedding_size, mask_zero=True)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(decoder_embedding, initial_state=encoder_state)

decoder_output = keras.layers.Dense(VOCAB_SIZE + 1, activation="softmax")(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

optimizer = keras.optimizers.Nadam()
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, None, 32)     655456      ['input_11[0][0]']               
                                                                                                  
 embedding_11 (Embedding)       (None, None, 32)     655488      ['input_12[0][0]']               
                                                                                            

In [75]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=30, validation_split=0.1, batch_size=512)

Epoch 1/30
126/126 [==============================] - 80s 578ms/step - loss: 3.8690 - accuracy: 0.0767 - val_loss: 3.7260 - val_accuracy: 0.0833
Epoch 2/30
126/126 [==============================] - 69s 548ms/step - loss: 3.4215 - accuracy: 0.0842 - val_loss: 3.3601 - val_accuracy: 0.0733
Epoch 3/30
126/126 [==============================] - 70s 553ms/step - loss: 3.1799 - accuracy: 0.0898 - val_loss: 3.2058 - val_accuracy: 0.0943
Epoch 4/30
126/126 [==============================] - 69s 547ms/step - loss: 2.9945 - accuracy: 0.1040 - val_loss: 2.9774 - val_accuracy: 0.1040
Epoch 5/30
126/126 [==============================] - 69s 546ms/step - loss: 2.6838 - accuracy: 0.1765 - val_loss: 2.6139 - val_accuracy: 0.1938
Epoch 6/30
126/126 [==============================] - 68s 544ms/step - loss: 2.2660 - accuracy: 0.3509 - val_loss: 2.1630 - val_accuracy: 0.4339
Epoch 7/30
126/126 [==============================] - 68s 539ms/step - loss: 1.8144 - accuracy: 0.5442 - val_loss: 1.7421 - val_ac

In [81]:
#lets train a little more to improve test accuracy
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10, validation_split=0.1, batch_size=512)

Epoch 1/10
126/126 [==============================] - 71s 567ms/step - loss: 0.0436 - accuracy: 0.9992 - val_loss: 0.4731 - val_accuracy: 0.9261
Epoch 2/10
126/126 [==============================] - 70s 553ms/step - loss: 0.0365 - accuracy: 0.9995 - val_loss: 0.4650 - val_accuracy: 0.9281
Epoch 3/10
126/126 [==============================] - 70s 553ms/step - loss: 0.0300 - accuracy: 0.9997 - val_loss: 0.4567 - val_accuracy: 0.9296
Epoch 4/10
126/126 [==============================] - 69s 546ms/step - loss: 0.0251 - accuracy: 0.9998 - val_loss: 0.4516 - val_accuracy: 0.9302
Epoch 5/10
126/126 [==============================] - 69s 549ms/step - loss: 0.0213 - accuracy: 0.9998 - val_loss: 0.4465 - val_accuracy: 0.9311
Epoch 6/10
126/126 [==============================] - 70s 555ms/step - loss: 0.0181 - accuracy: 0.9999 - val_loss: 0.4418 - val_accuracy: 0.9319
Epoch 7/10
126/126 [==============================] - 69s 552ms/step - loss: 0.0156 - accuracy: 0.9999 - val_loss: 0.4380 - val_ac

**almost 100% accuracy on training data after 30 epochs**

In [82]:
model.save(MODEL_PATH)

INFO:tensorflow:Assets written to: /models/nlp_models/assets


INFO:tensorflow:Assets written to: /models/nlp_models/assets


###**Simple seq2seq network**

**model evaluation on test data**

In [83]:
model = keras.models.load_model(MODEL_PATH)

In [93]:
ids = np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1)
ids, Y_test[:1]

(array([[ 1065,  1065,  1065,  1065,  1065,  1065,  1065,  1065,     0,
             2,   700,    32,   919,   221,   285,     7,  2486,  3909,
           241, 17219, 17056,   470, 17056,    43,  9330,     2,  4429,
             6,     2,  1101, 10902,  9330,   241,     9,    41,    45,
            50,  3282,  1455,    18]]),
 array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
             2,   700,    32,   919,   221,   285,     7,  2486,  5685,
           241,  7604, 18358,   470, 18359,    43, 18360,     2,  4429,
             6,     2,  1101,  9242,  5685,   241,     9,    41,    45,
            50,  3282,  1455,    18]], dtype=int32))

In [85]:
print("Evaluate on test data")
results = model.evaluate([X_test, X_test_decoder], Y_test, batch_size=512)
print("test loss, test acc:", results)

Evaluate on test data
35/35 [==============================] - 9s 207ms/step - loss: 0.4519 - accuracy: 0.9304
test loss, test acc: [0.45188069343566895, 0.9303902983665466]


###**Rebuild sentences**

In [104]:
def rebuild_test_sentences():
  Y_preds = model.predict([X_test[:10], X_test_decoder[:10]])
  encoded_argmax  = np.argmax(Y_preds, axis=-1)
  preds = tokenizer.sequences_to_texts(encoded_argmax[:10])
  return preds
 

rebuild_test_sentences()

['خواند خواند خواند خواند خواند خواند خواند خواند <UNK> در اوایل سال میلادی جاری زمانی که بنیاد شاخص\u200cهای جهانی شستشوی انفجارشدید ساله انفجارشدید وی هیمالیا در کامبوج را در فهرست کیفی هیمالیا جهانی این سازمان قرار داد مناقشات تشدید شد',
 'خواند خواند خواند خواند خواند خواند خواند خواند <UNK> در اوایل سال میلادی جاری زمانی که بنیاد پزشک جهانی جزئیاتی آث ساله انفجارشدید وی هیمالیا در کامبوج را در فهرست کیفی هیمالیا جهانی این سازمان قرار داد مناقشات تشدید شد',
 'خواند خواند خواند خواند خواند خواند خواند خواند <UNK> در اوایل سال میلادی جاری زمانی که بنیاد پزشک جهانی تکلیف انفجارشدید ساله انفجارشدید وی هیمالیا در کامبوج را در فهرست کیفی هیمالیا جهانی این سازمان قرار داد مناقشات تشدید شد',
 'خواند خواند خواند خواند خواند خواند خواند خواند <UNK> در اوایل سال میلادی جاری زمانی که بنیاد پزشک جهانی تکلیف انفجارشدید ساله انفجارشدید وی هیمالیا در کامبوج را در فهرست کیفی هیمالیا جهانی این سازمان قرار داد مناقشات تشدید شد',
 'خواند خواند خواند خواند خواند خواند خواند خواند <UNK> در اوایل سال میل